In [1]:
import pickle
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from transformers import AutoModel, AutoTokenizer
pd.set_option("max_colwidth", 200)

phobert = AutoModel.from_pretrained("vinai/phobert-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
def _load_pkl(path):
  with open(path, 'rb') as f:
    obj = pickle.load(f)
  return obj

# Load file


In [3]:
path_x = './file_pkl/padding.pkl'
path_y = './file_pkl/labels.pkl'
padded = _load_pkl(path_x)
labels = _load_pkl(path_y)

In [4]:
X_train, X_val, Y_train, Y_val = train_test_split(padded, labels, test_size=0.2)
X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size=0.1)

In [5]:
print(len(X_train))
print(len(X_val))
print(len(X_test))

4680
1300
520


# Mask
- Đánh dấu các từ thêm vào = 0 để không tính vào quá trình lấy features

In [6]:
# random user
index = 300

In [7]:
# get attention mask ( 0: not has word, 1: has word)
train_mask = np.where(X_train == 0, 0, 1)
val_mask = np.where(X_val == 0, 0, 1)
print('attention mask:', train_mask[index])
print('val_mask mask:', val_mask[index])

attention mask: [0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
val_mask mask: [0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [8]:
print(X_train.shape)
print(train_mask.shape)

(4680, 50)
(4680, 50)


In [9]:
train_text = torch.tensor(X_train).to(torch.long)
train_mask = torch.tensor(train_mask)
print(train_text)
print(train_mask)

tensor([[    0,   320,   574,  ...,     0,     0,     0],
        [    0,   574,   119,  ...,     0,     0,     0],
        [    0,    17, 28123,  ...,     0,     0,     0],
        ...,
        [    0,  2560,  2273,  ...,     0,     0,     0],
        [    0,   574,   119,  ...,     0,     0,     0],
        [    0,  2081,  5418,  ...,     0,     0,     0]])
tensor([[0, 1, 1,  ..., 0, 0, 0],
        [0, 1, 1,  ..., 0, 0, 0],
        [0, 1, 1,  ..., 0, 0, 0],
        ...,
        [0, 1, 1,  ..., 0, 0, 0],
        [0, 1, 1,  ..., 0, 0, 0],
        [0, 1, 1,  ..., 0, 0, 0]], dtype=torch.int32)


In [10]:
# Lấy features dầu ra từ BERT
with torch.no_grad():
    last_hidden_states = phobert(input_ids= train_text, attention_mask=train_mask)

v_features = last_hidden_states[0][:, 0, :].numpy()

In [11]:
def _save_pkl(path, obj):
  with open(path, 'wb') as f:
    pickle.dump(obj, f)

# Lưu lại các files
_save_pkl('./file_pkl/v_features.pkl', v_features)